In [2]:
import os

In [5]:
#path_to_scores = "runs_automatic_optimalv3_stemindex_nostemquery/"
#path_to_scores = "runs_automatic_optimalv3_nostem/"
path_to_scores = "runs_final/with_stemming/-originalidf/"
overleaf_output = []
for file in os.listdir(path_to_scores):
    if file[-6:] == "scores":
        if "covid" in file: continue

        # bm25, optimal
        #if not("bm25." in file and ".optimal." in file): 
        #    continue

        # bm25, non optimal
        #if not("wordnet." in file and "bm25." in file):
        #    continue

        # bm25s, optimal
        #if not("optimal." in file and "bm25s." in file):
        #    continue

        # bm25s, not optimal
        if not("wordnet." in file and "bm25s." in file):
           continue


        scores = []
        with open(path_to_scores + file, "r") as f:
            for line in f:
                if "all\t" not in line: continue
                scores.append(line.strip("\n").split())
        split_scores = file.split(".")
        disk = split_scores[1]
        if disk == "cran": continue
        queries = split_scores[2]
        source = split_scores[3]
        weight = split_scores[4]
        if split_scores[5] in ["1","5"]:
            weight += "." + split_scores[5]
            method = split_scores[6]
        else:
            method = split_scores[5]

        if "_" in weight:
            weight = "\_".join(weight.split("_"))
        map =  scores[0][2]
        p30 = scores[1][2]
        recall = scores[2][2]

        overleaf_output.append([disk,queries,source,weight,method,map,p30,recall])


sorted_overleaf_output = sorted(overleaf_output, reverse=True, key=lambda x: float(x[-1]))
sorted_overleaf_output = ["Disk & Queries & Expansion Source & Weighting & Method & MAP & P@10 & R@1000 \\\\"] + ["\\hline"] + [" & ".join(x) + "\\\\" for x in sorted_overleaf_output]
for line in sorted_overleaf_output:
    print(line)

Disk & Queries & Expansion Source & Weighting & Method & MAP & P@10 & R@1000 \\
\hline
disk12 & 51-100 & wordnet & contextual\_mean & bm25s & 0.1960 & 0.4680 & 0.4527\\
disk12 & 51-100 & wordnet & 0.5 & bm25s & 0.1927 & 0.4780 & 0.4448\\
disk12 & 51-100 & wordnet & contextual\_term & bm25s & 0.1923 & 0.4760 & 0.4429\\
disk12 & 51-100 & wordnet & wordnet\_wup & bm25s & 0.1936 & 0.4600 & 0.4409\\
disk12 & 51-100 & wordnet & wordnet\_path & bm25s & 0.1936 & 0.4600 & 0.4409\\
disk12 & 51-100 & wordnet & 0.1 & bm25s & 0.1920 & 0.4600 & 0.4378\\
disk12 & 51-100 & wordnet & 0 & bm25s & 0.1905 & 0.4620 & 0.4253\\
disk12 & 51-100 & wordnet & 1 & bm25s & 0.1783 & 0.4740 & 0.3967\\


In [5]:
base_path = "runs_final/without_stemming/"
idfs = ["-idfUnion/", "-originalidf/", "-pickAvgIDF/", "-pickLargerIDF/", "-pickSmallerIDF/"]
overleaf_output = {"covid": [], "cran": [], "disk12": []}
metric = -3 # recall = -1, p@30 = -2, map = -3

for idf in idfs:
    path_to_scores = base_path + idf
    for file in os.listdir(path_to_scores):

        #if "bm25s." not in file: continue


        if file[-6:] == "scores":

            # ignore all bm25 scores not in originalidf
            if "bm25." in file and idf != "-originalidf/": continue

            scores = []
            with open(path_to_scores + file, "r") as f:
                for line in f:
                    if "all\t" not in line: continue
                    scores.append(line.strip("\n").split())
            split_scores = file.split(".")
            disk = split_scores[1]
            if disk == "cran": continue

            queries = split_scores[2]
            source = split_scores[3]
            weight = split_scores[4]
            if split_scores[5] in ["1", "2","5"]:
                weight += "." + split_scores[5]
                method = split_scores[6]
            else:
                method = split_scores[5]

            if "_" in weight:
                weight = " ".join(weight.split("_"))
                #weight = "\_".join(weight.split("_"))
            try:
                map =  scores[0][2]
                p30 = scores[1][2]
                recall = scores[2][2]

                if weight == "norm across query":
                    weight = "Normalization"
                if source == "optimal":
                    source = "BERT"
                if method == "bm25s":
                    method = "TF-Merging"

                #if method == "TF-Merging" and idf[:-1] == "-originalidf": continue
                


                overleaf_output[disk].append([source,idf[:-1],weight,method,map,p30,recall])
                #overleaf_output[disk].append([disk,queries,source,idf[:-1],weight,method,map,p30,recall])
            except:
                print("ILL-FORMATTED SCORES: ", file, idf)

for disk in overleaf_output:
    sorted_overleaf_output = sorted(overleaf_output[disk], reverse=True, key=lambda x: float(x[metric]))
    sorted_overleaf_output = ["Disk & Queries & Expansion Source & IDF & Weighting & Method & MAP & P@10 & R@1000 \\\\"] + ["\\hline"] + [" & ".join(x) + "\\\\" for x in sorted_overleaf_output]
    print("Disk: ", disk)
    for line in sorted_overleaf_output:
        print(line)

Disk:  covid
Disk & Queries & Expansion Source & IDF & Weighting & Method & MAP & P@10 & R@1000 \\
\hline
BERT & -originalidf & Normalization & bm25 & 0.0278 & 0.0900 & 0.3096\\
BERT & -originalidf & 0.1 & bm25 & 0.0275 & 0.1133 & 0.3105\\
wordnet & -pickAvgIDF & contextual term & TF-Merging & 0.0265 & 0.1000 & 0.3085\\
wordnet & -pickSmallerIDF & contextual term & TF-Merging & 0.0265 & 0.1067 & 0.3085\\
BERT & -originalidf & 0 & bm25 & 0.0263 & 0.1133 & 0.3044\\
wordnet & -originalidf & 0 & bm25 & 0.0263 & 0.1133 & 0.3044\\
wordnet & -originalidf & 0.5 & TF-Merging & 0.0263 & 0.1033 & 0.3083\\
wordnet & -originalidf & contextual mean & TF-Merging & 0.0262 & 0.1067 & 0.3073\\
wordnet & -originalidf & contextual term & TF-Merging & 0.0262 & 0.1033 & 0.3085\\
wordnet & -pickAvgIDF & 0.5 & TF-Merging & 0.0262 & 0.1033 & 0.3083\\
wordnet & -pickAvgIDF & contextual mean & TF-Merging & 0.0262 & 0.1033 & 0.3073\\
BERT & -pickLargerIDF & 0.1 & TF-Merging & 0.0262 & 0.0867 & 0.3105\\
wordnet & 